In [ ]:
# cluster_crossmatch_vectorized.ipynb

import numpy as np
import h5py
from astropy.cosmology import Planck18, z_at_value
import astropy.units as u
import pandas as pd
from pymanticore.cosmology import ra_dec_to_galactic
import time

# Matching parameters
ANGULAR_SEP_TOLERANCE_DEG = 1  # degrees
REDSHIFT_TOLERANCE = 0.01

def vectorized_angular_separation_deg(lon1, lat1, lon2, lat2):
    """Vectorized angular separation calculation using Haversine formula"""
    # Convert to radians separately
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)
    
    # Broadcast to create matrices
    lon1 = lon1[:, np.newaxis]  # Shape: [N_manticore, 1]
    lat1 = lat1[:, np.newaxis]  # Shape: [N_manticore, 1] 
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return np.degrees(c)  # Shape: [N_manticore, N_external]

def vectorized_find_matches(manticore_data, external_data, ang_tol_deg, z_tol):
    """Vectorized matching between Manticore and external catalog"""
    
    # Calculate all pairwise angular separations
    ang_seps = vectorized_angular_separation_deg(
        manticore_data['gal_l'], manticore_data['gal_b'],
        external_data['gal_l'], external_data['gal_b']
    )
    
    # Calculate all pairwise redshift differences
    manticore_z = manticore_data['z'][:, np.newaxis]
    external_z = external_data['z']
    z_diffs = np.abs(external_z - manticore_z)
    
    # Apply matching criteria
    valid_matches = (ang_seps <= ang_tol_deg) & (z_diffs <= z_tol)
    
    # For each Manticore cluster, find the closest angular match among valid ones
    ang_seps_masked = np.where(valid_matches, ang_seps, np.inf)
    
    # Find best match index for each Manticore cluster
    best_match_indices = np.argmin(ang_seps_masked, axis=1)
    
    # Check if there were any valid matches
    has_match = np.any(valid_matches, axis=1)
    
    # Extract results
    n_manticore = len(manticore_data['gal_l'])
    
    matched_m500 = np.full(n_manticore, np.nan)
    matched_ang_sep = np.full(n_manticore, np.nan)
    matched_z_diff = np.full(n_manticore, np.nan)
    
    # Only fill in for clusters that have matches
    if np.any(has_match):
        valid_indices = np.where(has_match)[0]
        match_indices = best_match_indices[has_match]
        
        matched_m500[valid_indices] = external_data['m500'][match_indices]
        matched_ang_sep[valid_indices] = ang_seps[valid_indices, match_indices]
        matched_z_diff[valid_indices] = z_diffs[valid_indices, match_indices]
    
    return matched_m500, matched_ang_sep, matched_z_diff, has_match

In [ ]:
# Load Manticore posterior clusters
print("Loading Manticore clusters...")
fname = "/cosma7/data/dp004/rttw52/Manticore/new_analysis/clusters/temp/output/simplified_clusters.h5"

with h5py.File(fname, "r") as f:
    distances_mpc = f["clusters/median_properties/dist"][...]
    
    redshifts = []
    for dist in distances_mpc:
        z = z_at_value(Planck18.comoving_distance, dist * u.Mpc)
        redshifts.append(z)
    
    manticore_data = {
        "gal_l": f["clusters/median_properties/gal_l"][...],
        "gal_b": f["clusters/median_properties/gal_b"][...],
        "m500": f["clusters/median_properties/SO_500_crit_TotalMass"][...],
        "r500": f["clusters/median_properties/SO_500_crit_SORadius"][...],
        "dist": distances_mpc,
        "z": np.array(redshifts),
        "ra": f["clusters/median_properties/ra"][...],
        "dec": f["clusters/median_properties/dec"][...]
    }

print(f"Loaded {len(manticore_data['gal_l'])} Manticore clusters")

In [ ]:
# Load MCXC cluster catalog
print("Loading MCXC clusters...")
fname = "/cosma7/data/dp004/rttw52/Manticore/observational_data/mcxc_2_clusters/mcxc_clusters.hdf5"

with h5py.File(fname, "r") as f:
    _mcxc_data = f["all_clusters"][...]

_ra = np.degrees(_mcxc_data["phi"])
_dec = np.degrees(_mcxc_data["theta"])
gal_l, gal_b = ra_dec_to_galactic(_ra, _dec)

mcxc_data = {
    "gal_l": gal_l, 
    "gal_b": gal_b, 
    "z": _mcxc_data["z"], 
    "m500": _mcxc_data["M500"]*1e14, 
    "r500": _mcxc_data["R500"]
}

print(f"Loaded {len(mcxc_data['gal_l'])} MCXC clusters")

In [ ]:
# Load eROSITA cluster catalog
print("Loading eROSITA clusters...")
fname = "/cosma7/data/dp004/rttw52/Manticore/observational_data/erosita_clusters/erosita_clusters.hdf5"

with h5py.File(fname, "r") as f:
    _eros_data = f["all_clusters"][...]

_ra = np.degrees(_eros_data["phi"])
_dec = np.degrees(_eros_data["theta"])
gal_l, gal_b = ra_dec_to_galactic(_ra, _dec)

eros_data = {
    "gal_l": gal_l, 
    "gal_b": gal_b, 
    "z": _eros_data["z"], 
    "m500": _eros_data["M500"]*1e14, 
    "r500": _eros_data["R500"]
}

print(f"Loaded {len(eros_data['gal_l'])} eROSITA clusters")

In [ ]:
# Perform vectorized cross-matching
print("Cross-matching Manticore clusters with external catalogs...")
start_time = time.time()

# Match with MCXC
mcxc_m500, mcxc_ang_sep, mcxc_z_diff, mcxc_matched = vectorized_find_matches(
    manticore_data, mcxc_data, ANGULAR_SEP_TOLERANCE_DEG, REDSHIFT_TOLERANCE
)

# Match with eROSITA  
eros_m500, eros_ang_sep, eros_z_diff, eros_matched = vectorized_find_matches(
    manticore_data, eros_data, ANGULAR_SEP_TOLERANCE_DEG, REDSHIFT_TOLERANCE
)

end_time = time.time()
print(f"Matching completed in {end_time - start_time:.3f} seconds")

n_manticore = len(manticore_data['gal_l'])
print(f"MCXC matches: {np.sum(mcxc_matched)}/{n_manticore}")
print(f"eROSITA matches: {np.sum(eros_matched)}/{n_manticore}")
print(f"Both catalogs: {np.sum(mcxc_matched & eros_matched)}")

In [ ]:
# Create enhanced Manticore dataset with external matches
enhanced_manticore = {
    # Original Manticore data
    "gal_l": manticore_data["gal_l"],
    "gal_b": manticore_data["gal_b"],
    "z": manticore_data["z"],
    "m500_manticore": manticore_data["m500"],
    "r500": manticore_data["r500"],
    "ra": manticore_data["ra"],
    "dec": manticore_data["dec"],
    
    # MCXC matches
    "m500_mcxc": mcxc_m500,
    "mcxc_ang_sep": mcxc_ang_sep,
    "mcxc_z_diff": mcxc_z_diff,
    "mcxc_matched": mcxc_matched,
    
    # eROSITA matches
    "m500_erosita": eros_m500,
    "eros_ang_sep": eros_ang_sep,
    "eros_z_diff": eros_z_diff,
    "eros_matched": eros_matched
}

print("\nEnhanced Manticore dataset created with external catalog matches")
print(f"Total clusters: {len(enhanced_manticore['gal_l'])}")
print(f"With MCXC counterparts: {np.sum(enhanced_manticore['mcxc_matched'])}")
print(f"With eROSITA counterparts: {np.sum(enhanced_manticore['eros_matched'])}")

In [ ]:
# Summary statistics
print(f"\nMatching Statistics (tolerance: {ANGULAR_SEP_TOLERANCE_DEG}° angular, Δz={REDSHIFT_TOLERANCE})")
print("="*70)

mcxc_matches = np.sum(enhanced_manticore['mcxc_matched'])
eros_matches = np.sum(enhanced_manticore['eros_matched'])
both_matches = np.sum(enhanced_manticore['mcxc_matched'] & enhanced_manticore['eros_matched'])
any_matches = np.sum(enhanced_manticore['mcxc_matched'] | enhanced_manticore['eros_matched'])

print(f"MCXC matches:     {mcxc_matches:4d} ({100*mcxc_matches/n_manticore:.1f}%)")
print(f"eROSITA matches:  {eros_matches:4d} ({100*eros_matches/n_manticore:.1f}%)")
print(f"Both catalogs:    {both_matches:4d} ({100*both_matches/n_manticore:.1f}%)")
print(f"Either catalog:   {any_matches:4d} ({100*any_matches/n_manticore:.1f}%)")
print(f"No matches:       {n_manticore-any_matches:4d} ({100*(n_manticore-any_matches)/n_manticore:.1f}%)")

if mcxc_matches > 0:
    print(f"\nMCXC match quality:")
    print(f"  Median angular sep: {np.nanmedian(enhanced_manticore['mcxc_ang_sep']):.4f}°")
    print(f"  Median redshift diff: {np.nanmedian(enhanced_manticore['mcxc_z_diff']):.4f}")

if eros_matches > 0:
    print(f"\neROSITA match quality:")
    print(f"  Median angular sep: {np.nanmedian(enhanced_manticore['eros_ang_sep']):.4f}°")
    print(f"  Median redshift diff: {np.nanmedian(enhanced_manticore['eros_z_diff']):.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Create plot of angular separation vs distance for matches
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# MCXC matches
mcxc_mask = enhanced_manticore['mcxc_matched']
if np.any(mcxc_mask):
    ax.scatter(manticore_data['dist'][mcxc_mask], 
               enhanced_manticore['mcxc_ang_sep'][mcxc_mask],
               color='blue', alpha=0.7, s=30, label=f'MCXC (N={np.sum(mcxc_mask)})',
               marker='o')

# eROSITA matches  
eros_mask = enhanced_manticore['eros_matched']
if np.any(eros_mask):
    ax.scatter(manticore_data['dist'][eros_mask],
               enhanced_manticore['eros_ang_sep'][eros_mask], 
               color='red', alpha=0.7, s=30, label=f'eROSITA (N={np.sum(eros_mask)})',
               marker='s')

# Formatting
ax.set_xlabel('Manticore Cluster Distance [Mpc]', fontsize=12)
ax.set_ylabel('Angular Separation [degrees]', fontsize=12)
ax.set_title('Cross-match Angular Separation vs Distance', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend()

# Add horizontal line at tolerance
ax.axhline(ANGULAR_SEP_TOLERANCE_DEG, color='black', linestyle='--', 
           alpha=0.5, label=f'Tolerance ({ANGULAR_SEP_TOLERANCE_DEG}°)')

# Set reasonable axis limits
if np.any(mcxc_mask) or np.any(eros_mask):
    all_distances = np.concatenate([
        manticore_data['dist'][mcxc_mask] if np.any(mcxc_mask) else [],
        manticore_data['dist'][eros_mask] if np.any(eros_mask) else []
    ])
    if len(all_distances) > 0:
        ax.set_xlim(np.min(all_distances) * 0.9, np.max(all_distances) * 1.1)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Create mass bins for median calculation
mass_bins = np.logspace(13.5, 15.5, 15)
mass_centers = np.sqrt(mass_bins[:-1] * mass_bins[1:])

# MCXC matches
mcxc_mask = enhanced_manticore['mcxc_matched']
if np.any(mcxc_mask):
    mcxc_ratio = enhanced_manticore['m500_manticore'][mcxc_mask] / enhanced_manticore['m500_mcxc'][mcxc_mask]
    mcxc_masses = enhanced_manticore['m500_manticore'][mcxc_mask]
    
    # Scatter with faded points
    ax.scatter(mcxc_masses, mcxc_ratio,
               color='blue', alpha=0.2, s=20, marker='o')
    
    # Calculate binned statistics
    mcxc_medians = []
    mcxc_p16 = []
    mcxc_p84 = []
    valid_centers = []
    
    for i in range(len(mass_bins)-1):
        in_bin = (mcxc_masses >= mass_bins[i]) & (mcxc_masses < mass_bins[i+1])
        if np.sum(in_bin) >= 3:  # Need at least 3 points
            bin_ratios = mcxc_ratio[in_bin]
            mcxc_medians.append(np.median(bin_ratios))
            mcxc_p16.append(np.percentile(bin_ratios, 16))
            mcxc_p84.append(np.percentile(bin_ratios, 84))
            valid_centers.append(mass_centers[i])
    
    if len(mcxc_medians) > 0:
        mcxc_medians = np.array(mcxc_medians)
        mcxc_p16 = np.array(mcxc_p16)
        mcxc_p84 = np.array(mcxc_p84)
        valid_centers = np.array(valid_centers)
        
        # Plot median line and percentile spread
        ax.plot(valid_centers, mcxc_medians, color='blue', linewidth=2, 
                label=f'MCXC median (N={np.sum(mcxc_mask)})')
        ax.fill_between(valid_centers, mcxc_p16, mcxc_p84, 
                        color='blue', alpha=0.3, label='MCXC 16-84%')

# eROSITA matches  
eros_mask = enhanced_manticore['eros_matched']
if np.any(eros_mask):
    eros_ratio = enhanced_manticore['m500_manticore'][eros_mask] / enhanced_manticore['m500_erosita'][eros_mask]
    eros_masses = enhanced_manticore['m500_manticore'][eros_mask]
    
    # Scatter with faded points
    ax.scatter(eros_masses, eros_ratio,
               color='red', alpha=0.2, s=20, marker='s')
    
    # Calculate binned statistics
    eros_medians = []
    eros_p16 = []
    eros_p84 = []
    valid_centers_eros = []
    
    for i in range(len(mass_bins)-1):
        in_bin = (eros_masses >= mass_bins[i]) & (eros_masses < mass_bins[i+1])
        if np.sum(in_bin) >= 3:  # Need at least 3 points
            bin_ratios = eros_ratio[in_bin]
            eros_medians.append(np.median(bin_ratios))
            eros_p16.append(np.percentile(bin_ratios, 16))
            eros_p84.append(np.percentile(bin_ratios, 84))
            valid_centers_eros.append(mass_centers[i])
    
    if len(eros_medians) > 0:
        eros_medians = np.array(eros_medians)
        eros_p16 = np.array(eros_p16)
        eros_p84 = np.array(eros_p84)
        valid_centers_eros = np.array(valid_centers_eros)
        
        # Plot median line and percentile spread
        ax.plot(valid_centers_eros, eros_medians, color='red', linewidth=2,
                label=f'eROSITA median (N={np.sum(eros_mask)})')
        ax.fill_between(valid_centers_eros, eros_p16, eros_p84,
                        color='red', alpha=0.3, label='eROSITA 16-84%')

# Formatting
ax.set_xlabel('Manticore M500 [M☉]', fontsize=12)
ax.set_ylabel('M500 Ratio (Manticore/External)', fontsize=12)
ax.set_title('Mass Ratio: Manticore vs External Catalogs', fontsize=14)
ax.set_xscale('log')
ax.grid(True, alpha=0.3)
ax.set_ylim(-1,10)
ax.legend()

# Add horizontal line at 1:1 ratio
ax.axhline(1.0, color='black', linestyle='--', alpha=0.5, label='1:1 ratio')

plt.tight_layout()
plt.show()